# Minibatch training

## Data

In [1]:
# Download the MNIST dataset
import gzip
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
from pathlib import Path
from urllib.request import urlretrieve

MNIST_URL = "https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true"

data_path = Path("data/mnist")
data_path.mkdir(exist_ok=True)
gz_path = data_path / "mnist.pkl.gz"

mpl.rcParams["image.cmap"] = "gray"

if not gz_path.exists():
    urlretrieve(MNIST_URL, gz_path)

# File contains a tuple of tuples for the x and y, train and validation data
# Images are 28x28
with gzip.open(gz_path, "rb") as file:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(file, encoding="latin-1")

# Put into tensors
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Model

In [2]:
# Lets define some constants that describe the data
n_examples, n_pixels = x_train.shape
possible_values = y_train.max() + 1

# How many nodes/activations/line thingys
n_hidden = 50

n_examples, n_pixels, possible_values

(50000, 784, tensor(10))

In [3]:
from torch import nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, n_hidden), nn.ReLU(), nn.Linear(n_hidden, n_out)]

    def __call__(self, inp):
        res = inp
        for layer in self.layers:
            res = layer(res)

        return res

In [4]:
model = Model(n_pixels, n_hidden, 10)
pred = model(x_train)
pred.shape

torch.Size([50000, 10])

## Cross entropy loss

We need to improve our loss function as MSE doesn't make sense, the distance between incorrecd predictions doesn't indicate how good/bad the prediction is. For example if the target is 2, then 3 isnt a better guess than 9.

We are outputting a pred for each possible number. As only one is possible at a time our targets are a one hot encoded matrix. If our targets are going to sum to 1, then it makes sense that our preds do too. We can calculate the softmax for each pred. Then for our loss func we compare the softmaxes to the 1 hot encoded targets. As we are 1 hot encoded we can ignore the other targets and just take the log.

In [5]:
def log_softmax(x):
    return (x.exp() / (x.exp().sum(-1, keepdim=True))).log()


log_softmax(pred)

tensor([[-2.3682, -2.2036, -2.3934,  ..., -2.3103, -2.3528, -2.3528],
        [-2.3118, -2.2293, -2.3477,  ..., -2.4072, -2.3332, -2.4887],
        [-2.3107, -2.3083, -2.2292,  ..., -2.2186, -2.3733, -2.2416],
        ...,
        [-2.3486, -2.1997, -2.2944,  ..., -2.2555, -2.3861, -2.3415],
        [-2.3534, -2.1716, -2.3589,  ..., -2.3260, -2.3768, -2.3212],
        [-2.3240, -2.2793, -2.3415,  ..., -2.3025, -2.4082, -2.3045]],
       grad_fn=<LogBackward0>)

In [6]:
# As log(a/b) = log(a) - log(b), we can simplify things to:


def log_softmax(x):
    return x - x.exp().sum(-1, keepdim=True).log()


log_softmax(pred)

tensor([[-2.3682, -2.2036, -2.3934,  ..., -2.3103, -2.3528, -2.3528],
        [-2.3118, -2.2293, -2.3477,  ..., -2.4072, -2.3332, -2.4887],
        [-2.3107, -2.3083, -2.2292,  ..., -2.2186, -2.3733, -2.2416],
        ...,
        [-2.3486, -2.1997, -2.2944,  ..., -2.2555, -2.3861, -2.3415],
        [-2.3534, -2.1716, -2.3589,  ..., -2.3260, -2.3768, -2.3212],
        [-2.3240, -2.2793, -2.3415,  ..., -2.3025, -2.4082, -2.3045]],
       grad_fn=<SubBackward0>)

In [7]:
# Its possible for the sum of the exponentials of big activations to overflow
# pytorch uses some tricks to solve this for use in one function
def log_softmax(x):
    return x - x.logsumexp(-1, keepdim=True)


sm_pred = log_softmax(pred)
pred

tensor([[-0.0130,  0.1516, -0.0382,  ...,  0.0449,  0.0023,  0.0023],
        [ 0.0831,  0.1656,  0.0471,  ..., -0.0124,  0.0617, -0.0939],
        [ 0.0607,  0.0631,  0.1422,  ...,  0.1528, -0.0019,  0.1298],
        ...,
        [-0.0156,  0.1332,  0.0385,  ...,  0.0775, -0.0532, -0.0085],
        [-0.0248,  0.1570, -0.0303,  ...,  0.0026, -0.0482,  0.0074],
        [-0.0280,  0.0167, -0.0455,  ..., -0.0066, -0.1123, -0.0085]],
       grad_fn=<AddmmBackward0>)

In [8]:
# How do we index into the preds we want
# The y values tell us targets, which are also the index
y_train[:5]

tensor([5, 0, 4, 1, 9])

In [9]:
# So for each i we want row i and col y_train[i]. eg for pred at row 0
sm_pred[0, y_train[0]]

tensor(-2.3057, grad_fn=<SelectBackward0>)

In [10]:
# We can do this for all of them like this
sm_pred[range(y_train.shape[0]), y_train]

tensor([-2.3057, -2.3118, -2.4710,  ..., -2.3861, -2.3694, -2.4082],
       grad_fn=<IndexBackward0>)

In [11]:
# So our cross entropy loss function looks like this
def nll(inp, target):
    return -inp[range(target.shape[0]), target].mean()


loss = nll(sm_pred, y_train)
loss

tensor(2.2933, grad_fn=<NegBackward0>)

In [12]:
# pytorch gives us both of these functions, NLL = negative log likelihood
F.nll_loss(F.log_softmax(pred, -1), y_train)

tensor(2.2933, grad_fn=<NllLossBackward0>)

In [13]:
# And a cross entropy function to do it all in one
F.cross_entropy(pred, y_train)

tensor(2.2933, grad_fn=<NllLossBackward0>)

## Training loop

In [14]:
loss_func = F.cross_entropy
batch_size = 64

# First we would run a minibatch
mini_batch = x_train[0:batch_size]
mini_batch_y = y_train[0:batch_size]

preds = model(mini_batch)
preds[0], preds.shape

(tensor([-0.0130,  0.1516, -0.0382,  0.2053, -0.0021,  0.0495,  0.0950,  0.0449,
          0.0023,  0.0023], grad_fn=<SelectBackward0>),
 torch.Size([64, 10]))

In [15]:
# Then we would calculate the loss
loss_func(preds, mini_batch_y)

tensor(2.2939, grad_fn=<NllLossBackward0>)

In [16]:
# For each of our predictions what did we predict, we need the highest number from each set of preds and get its index
preds.argmax(dim=1)

tensor([3, 3, 3, 1, 1, 6, 6, 3, 3, 6, 1, 6, 1, 3, 6, 6, 3, 5, 3, 1, 3, 3, 6, 1,
        3, 3, 1, 1, 1, 1, 3, 1, 3, 5, 5, 6, 3, 3, 1, 6, 1, 3, 1, 6, 3, 6, 3, 1,
        6, 3, 3, 6, 1, 6, 6, 6, 5, 6, 6, 1, 7, 3, 3, 5])

In [17]:
# We can see which we got correct
preds.argmax(dim=1) == mini_batch_y

tensor([False, False, False,  True, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False,  True,
         True, False, False, False,  True, False, False, False, False,  True,
         True, False, False, False, False, False, False, False, False,  True,
        False, False, False, False])

In [18]:
# We can use it to calculate accuracy, this isnt needed for the NN but helps us understand whats going on
def accuracy(out, targets):
    return (out.argmax(dim=1) == targets).float().mean()


accuracy(preds, mini_batch_y)

tensor(0.1562)

In [19]:
# Now we want to do all of the batches for a bunch of epochs, updating the weights each time
import torch

lr = 0.5
epochs = 5

for epoch in range(epochs):
    for batch_start in range(0, n_examples, batch_size):
        # Get the slice
        sl = slice(batch_start, min(n_examples, batch_start + batch_size))

        batch_inp = x_train[sl]
        batch_targets = y_train[sl]

        # Run the model
        preds = model(batch_inp)
        loss = loss_func(preds, batch_targets)

        if batch_start == 0:
            print("loss:", loss.item(), "acc: ", accuracy(preds, batch_targets).item())

        loss.backward()

        # Update the weights
        with torch.no_grad():
            for layer in model.layers:
                if hasattr(layer, "weight"):
                    layer.weight -= layer.weight.grad * lr
                    layer.bias -= layer.bias.grad * lr
                    layer.weight.grad.zero_()
                    layer.bias.grad.zero_()

loss: 2.2939493656158447 acc:  0.15625
loss: 0.18433193862438202 acc:  0.9375
loss: 0.17788918316364288 acc:  0.953125
loss: 0.13251781463623047 acc:  0.953125
loss: 0.0965082049369812 acc:  0.96875


## Parameters and optim

### Parameters

In [20]:
m1 = nn.Module()
m1.foo = nn.Linear(3, 4)

# Notice that the Module can automatically track all of the parameters in the
# layer that is assigned to it, how does that work?
m1, list(m1.named_children()), list(m1.parameters())

(Module(
   (foo): Linear(in_features=3, out_features=4, bias=True)
 ),
 [('foo', Linear(in_features=3, out_features=4, bias=True))],
 [Parameter containing:
  tensor([[ 0.3047,  0.2764,  0.5723],
          [ 0.4992,  0.0261,  0.5466],
          [-0.0185, -0.4649,  0.3647],
          [ 0.3825,  0.0715, -0.5044]], requires_grad=True),
  Parameter containing:
  tensor([-0.3105, -0.2362, -0.4593,  0.1845], requires_grad=True)])

In [21]:
class MLP(nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_out)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.l2(self.relu(self.l1(x)))


# We normally use Module by inheriting from it and assigning it a bynch of layers

model = MLP(n_pixels, n_hidden, 10)
model, model.l1, list(model.named_children()), list(map(lambda x: x.shape, model.parameters()))

(MLP(
   (l1): Linear(in_features=784, out_features=50, bias=True)
   (l2): Linear(in_features=50, out_features=10, bias=True)
   (relu): ReLU()
 ),
 Linear(in_features=784, out_features=50, bias=True),
 [('l1', Linear(in_features=784, out_features=50, bias=True)),
  ('l2', Linear(in_features=50, out_features=10, bias=True)),
  ('relu', ReLU())],
 [torch.Size([50, 784]),
  torch.Size([50]),
  torch.Size([10, 50]),
  torch.Size([10])])

In [22]:
# So already we can rewrite our loop knowing we can just get all of the params from the model


def fit():
    for epoch in range(epochs):
        for batch_start in range(0, n_examples, batch_size):
            # Get the slice
            sl = slice(batch_start, min(n_examples, batch_start + batch_size))

            batch_inp = x_train[sl]
            batch_targets = y_train[sl]

            # Run the model
            preds = model(batch_inp)
            loss = loss_func(preds, batch_targets)

            if batch_start == 0:
                print("loss:", loss.item(), "acc: ", accuracy(preds, batch_targets).item())

            loss.backward()

            # Update the weights
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()


fit()

loss: 2.295954704284668 acc:  0.109375
loss: 0.10442031174898148 acc:  0.953125
loss: 0.11948614567518234 acc:  0.96875
loss: 0.10430541634559631 acc:  0.96875
loss: 0.10337503999471664 acc:  0.953125


In [23]:
# Here's how we would implement this if nn.Module didn't exist
class MyModule:
    def __init__(self, n_in, n_hidden, n_out):
        self._modules = {}
        self.l1 = nn.Linear(n_in, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_out)

    # We hook into setattr and update _modules when some sets something new
    def __setattr__(self, key, value):
        if not key.startswith("_"):
            self._modules[key] = value
        super().__setattr__(key, value)

    # Print modules
    def __repr__(self):
        return f"{self._modules}"

    # Generate the parameters of every module
    def parameters(self):
        for mod in self.modules.values():
            for p in mod.parameters():
                yield p


myM = MyModule(n_pixels, n_hidden, 10)
myM

{'l1': Linear(in_features=784, out_features=50, bias=True), 'l2': Linear(in_features=50, out_features=10, bias=True)}

### Registering modules

We previously stored all our layers in a .layers member. How would we do that with nn.module. You can manually add layers using `add_module`.

In [24]:
from functools import reduce


class Model(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers

        for idx, layer in enumerate(self.layers):
            self.add_module(f"layer_{idx}", layer)

    def __call__(self, x):
        return reduce(lambda val, layer: layer(val), self.layers, x)


layers = [nn.Linear(n_pixels, n_hidden), nn.ReLU(), nn.Linear(n_hidden, 10)]
model = Model(layers)

model

Model(
  (layer_0): Linear(in_features=784, out_features=50, bias=True)
  (layer_1): ReLU()
  (layer_2): Linear(in_features=50, out_features=10, bias=True)
)

#### nn.ModuleList

`nn.ModuleList` does this.

In [25]:
class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


model = SequentialModel(layers)
model

SequentialModel(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [26]:
fit()

loss: 2.316998243331909 acc:  0.125
loss: 0.17858779430389404 acc:  0.9375
loss: 0.15072037279605865 acc:  0.953125
loss: 0.17366889119148254 acc:  0.96875
loss: 0.1267264485359192 acc:  0.96875


#### nn.Sequential

`nn.Sequential` already exists and does the lot for us.

In [27]:
layers = [nn.Linear(n_pixels, n_hidden), nn.ReLU(), nn.Linear(n_hidden, 10)]
model = nn.Sequential(*layers)
model

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)

In [28]:
fit()

loss: 2.315805196762085 acc:  0.0625
loss: 0.1865747720003128 acc:  0.9375
loss: 0.08276090770959854 acc:  0.96875
loss: 0.08583660423755646 acc:  0.96875
loss: 0.03434313088655472 acc:  1.0


### Optimizer

In [29]:
# We could put our optimisation steps into a class


class Optimizer:
    def __init__(self, params, lr=0.5):
        self.params = list(params)
        self.lr = lr

    def step(self):
        with torch.no_grad():
            for p in self.params:
                p -= p.grad * self.lr

    def zero_grad(self):
        for p in self.params:
            p.grad.data.zero_()

In [30]:
layers = [nn.Linear(n_pixels, n_hidden), nn.ReLU(), nn.Linear(n_hidden, 10)]
model = nn.Sequential(*layers)
opt = Optimizer(model.parameters())


# And now our training loop will look like this
def fit():
    for epoch in range(epochs):
        for batch_start in range(0, n_examples, batch_size):
            # Get the slice
            sl = slice(batch_start, min(n_examples, batch_start + batch_size))

            batch_inp = x_train[sl]
            batch_targets = y_train[sl]

            # Run the model
            preds = model(batch_inp)
            loss = loss_func(preds, batch_targets)

            if batch_start == 0:
                print("loss:", loss.item(), "acc: ", accuracy(preds, batch_targets).item())

            loss.backward()

            # Update the weights
            opt.step()
            opt.zero_grad()


fit()

loss: 2.297377109527588 acc:  0.0625
loss: 0.18612894415855408 acc:  0.9375
loss: 0.15381328761577606 acc:  0.9375
loss: 0.13656480610370636 acc:  0.9375
loss: 0.09828701615333557 acc:  0.96875


In [31]:
# pytorch gives use this as well
from torch import optim


def get_model():
    layers = [nn.Linear(n_pixels, n_hidden), nn.ReLU(), nn.Linear(n_hidden, 10)]
    model = nn.Sequential(*layers)
    opt = optim.SGD(model.parameters(), lr=lr)

    return model, opt


model, opt = get_model()


def fit():
    for epoch in range(epochs):
        for batch_start in range(0, n_examples, batch_size):
            # Get the slice
            sl = slice(batch_start, min(n_examples, batch_start + batch_size))

            batch_inp = x_train[sl]
            batch_targets = y_train[sl]

            # Run the model
            preds = model(batch_inp)
            loss = loss_func(preds, batch_targets)

            if batch_start == 0:
                print("loss:", loss.item(), "acc: ", accuracy(preds, batch_targets).item())

            loss.backward()

            # Update the weights
            opt.step()
            opt.zero_grad()


fit()

loss: 2.31352162361145 acc:  0.109375
loss: 0.14608719944953918 acc:  0.953125
loss: 0.0877109169960022 acc:  0.96875
loss: 0.05135871097445488 acc:  0.96875
loss: 0.0472152940928936 acc:  0.96875


## Dataset and DataLoader

What if we want to more easily iterate through minimatches and x/y.

In [32]:
class Dataset:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return self.x[i], self.y[i]


train_ds = Dataset(x_train, y_train)
valid_ds = Dataset(x_valid, y_valid)

len(train_ds), len(valid_ds)

(50000, 10000)

In [33]:
# __getitem__ works with tensors
train_ds[0:5], train_ds[0:5][0].shape, train_ds[0:5][1].shape

((tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  tensor([5, 0, 4, 1, 9])),
 torch.Size([5, 784]),
 torch.Size([5]))

In [34]:
# We can now simplify the slicing


def fit():
    for epoch in range(epochs):
        for batch_start in range(0, n_examples, batch_size):
            # Get the slice
            sl = slice(batch_start, min(n_examples, batch_start + batch_size))

            batch_inp, batch_targets = train_ds[batch_start : min(n_examples, batch_start + batch_size)]

            # Run the model
            preds = model(batch_inp)
            loss = loss_func(preds, batch_targets)

            if batch_start == 0:
                print("loss:", loss.item(), "acc: ", accuracy(preds, batch_targets).item())

            loss.backward()

            # Update the weights
            opt.step()
            opt.zero_grad()


model, opt = get_model()
fit()

loss: 2.3270351886749268 acc:  0.078125
loss: 0.15340012311935425 acc:  0.9375
loss: 0.15084929764270782 acc:  0.921875
loss: 0.1238880455493927 acc:  0.9375
loss: 0.08751741796731949 acc:  0.953125


In [35]:
# A DataLoader is an iterator that will help us with looping over the minibatches


class DataLoader:
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        for batch_start in range(0, len(self.dataset), batch_size):
            yield self.dataset[batch_start : batch_start + self.batch_size]


train_dl = DataLoader(train_ds, batch_size)
valid_dl = DataLoader(valid_ds, batch_size)

In [36]:
# Now we can remove the slicing form our loop
def fit():
    for epoch in range(epochs):
        for batch_inp, batch_targets in train_dl:
            # Run the model
            preds = model(batch_inp)
            loss = loss_func(preds, batch_targets)

            loss.backward()

            # Update the weights
            opt.step()
            opt.zero_grad()
        print("loss:", loss.item(), "acc: ", accuracy(preds, batch_targets).item())


model, opt = get_model()
fit()

loss: 0.5205250382423401 acc:  0.875
loss: 0.26634106040000916 acc:  0.875
loss: 0.08028078824281693 acc:  1.0
loss: 0.02806195430457592 acc:  1.0
loss: 0.036086585372686386 acc:  1.0


## Random sampling

What if we want our training set to be in a random order that differs every iteration (but keep the validations set the ssame)?

In [37]:
import random


# Iterates over indices that  may or may not be shuffled
class Sampler:
    def __init__(self, dataset, shuffle=False):
        self.N = len(dataset)
        self.shuffle = shuffle

    def __iter__(self):
        # List indices for the dataset
        indices = list(range(self.N))

        if self.shuffle:
            random.shuffle(indices)

        return iter(indices)


list(Sampler(train_ds))[:5], list(Sampler(train_ds, True))[:5]

([0, 1, 2, 3, 4], [8989, 15354, 49658, 32012, 39419])

In [38]:
import fastcore.all as fc


# Gets batches of the indices given by a sampler
class BatchSampler:
    def __init__(self, sampler, batch_size, drop_last=False):
        fc.store_attr()  # Stores all inputs as members with same name

    def __iter__(self):
        for chunk in fc.chunked(iter(self.sampler), self.batch_size, drop_last=self.drop_last):
            yield chunk


# Batches of 2, randomised
list(BatchSampler(Sampler(train_ds, True), 2))[:5]

[[20896, 14995], [3185, 3092], [33569, 38996], [19244, 35012], [25790, 27146]]

In [39]:
# We can update the DataLoader to use a BatchSampler rather than being told a batch size


# We need a collation function to stack all of the Xs and all of the Ys together into tensors
def collate(data):
    data_x, data_y = zip(*data)
    return torch.stack(data_x), torch.stack(data_y)


class DataLoader:
    def __init__(self, dataset, batch_sampler, collate_fn=collate):
        fc.store_attr()

    def __iter__(self):
        yield from (self.collate_fn(self.dataset[i] for i in b) for b in self.batch_sampler)


train_samp = BatchSampler(Sampler(train_ds, shuffle=True), batch_size)
valid_samp = BatchSampler(Sampler(valid_ds, shuffle=False), batch_size)

train_dl = DataLoader(train_ds, batch_sampler=train_samp)
valid_dl = DataLoader(valid_ds, batch_sampler=valid_samp)

batch = next(iter(train_dl))
batch, batch[0].shape, batch[1].shape

((tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  tensor([1, 0, 3, 7, 8, 1, 9, 6, 5, 8, 4, 2, 3, 2, 2, 1, 3, 7, 5, 2, 8, 7, 6, 4,
          4, 1, 8, 6, 1, 9, 2, 1, 2, 0, 2, 8, 4, 3, 3, 6, 2, 5, 8, 4, 6, 3, 4, 7,
          5, 7, 5, 7, 4, 1, 2, 3, 5, 2, 1, 6, 9, 6, 2, 2])),
 torch.Size([64, 784]),
 torch.Size([64]))

In [40]:
model, opt = get_model()
fit()

loss: 0.05326978117227554 acc:  1.0
loss: 0.12145218253135681 acc:  0.9375
loss: 0.11459718644618988 acc:  0.9375
loss: 0.15202879905700684 acc:  0.9375
loss: 0.01949377916753292 acc:  1.0


### Multiprocessing DataLoader

What if we want to run this in parallel to speed things up.

In [41]:
# We want to be able to process something a bit like this but in parallel
for o in map(train_ds.__getitem__, ([3, 6], [8, 1])):
    print(o)

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([1, 1]))
(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([1, 0]))


In [42]:
# Pytorch gives us all of these and supports multiprocessing
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler, BatchSampler

train_samp = BatchSampler(RandomSampler(train_ds), batch_size, drop_last=False)
valid_samp = BatchSampler(SequentialSampler(valid_ds), batch_size, drop_last=False)

train_dl = DataLoader(train_ds, batch_sampler=train_samp, collate_fn=collate)
valid_dl = DataLoader(valid_ds, batch_sampler=valid_samp, collate_fn=collate)

model, opt = get_model()
fit()

loss: 0.1121179386973381 acc:  0.875
loss: 0.21432000398635864 acc:  0.9375
loss: 0.019699424505233765 acc:  1.0
loss: 0.19220580160617828 acc:  0.875
loss: 0.28950852155685425 acc:  0.875


In [43]:
# We can also pass a batch sampler as a sampler as we are able to index multiple things at once with no collate
# pytorch kust autogens a BatchSampler for us
train_dl = DataLoader(train_ds, sampler=train_samp)
valid_dl = DataLoader(valid_ds, sampler=valid_samp)

# As random sampling is so common, we can also just pass shuffle flags, and that dataset
train_dl = DataLoader(train_ds, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, shuffle=False)

model, opt = get_model()
fit()

loss: 2.525796413421631 acc:  0.0
loss: 3.250904083251953 acc:  0.0
loss: 2.5235815048217773 acc:  0.0
loss: 2.464507579803467 acc:  0.0
loss: 2.1057496070861816 acc:  0.0


## Validation

We've not done anything with the validation set yet. Lets update fit to take in a validation set. We'll print the validation loss for each epoch.

In [48]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Some layers behave differently during training and validation so we have to tell them
        model.train()
        for batch_inp, batch_targets in train_dl:
            # Run the model
            preds = model(batch_inp)
            loss = loss_func(preds, batch_targets)

            loss.backward()

            # Update the weights
            opt.step()
            opt.zero_grad()

        # Now run the validation set
        model.eval()
        with torch.no_grad():
            total_loss = 0.0
            total_acc = 0.0
            count = 0

            for batch_inp, batch_targets in train_dl:
                preds = model(batch_inp)

                count += len(batch_inp)
                total_loss += loss_func(preds, batch_targets).item() * len(batch_inp)
                total_acc += accuracy(preds, batch_targets).item() * len(batch_inp)

        total_loss /= count
        total_acc /= count
        print(f"epoch: {epoch}, loss: {total_loss}, acc: {total_acc}")

    return total_loss, total_acc


def get_dls(train_ds, valid_ds, batch_size, **kwargs):
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True, **kwargs),
        DataLoader(valid_ds, batch_size=batch_size * 2, **kwargs),
    )


train_dl, valid_dl = get_dls(train_ds, valid_ds, 64)
model, opt = get_model()
loss, acc = fit(5, model, F.cross_entropy, opt, train_dl, valid_dl)

epoch: 0, loss: 0.16989297520160676, acc: 0.94822
epoch: 1, loss: 0.21148953001499177, acc: 0.93568
epoch: 2, loss: 0.09376879837676882, acc: 0.9701
epoch: 3, loss: 0.07792397175550461, acc: 0.97562
epoch: 4, loss: 0.05999621275998652, acc: 0.98044
